In [1]:
import random
import collections
import numpy as np

## Part 1: Keys Concepts of Private Heavy Hitters via Federated Analytics

#### a) Aggregate = Count

In [2]:
# Simulate local data at each bank
banks_data = [
    ['abc12345', 'abc67890', 'xyz11111'], #bank A
    ['abc22222', 'def33333', 'abc12345'], #bank B
    ['xyz99999', 'abc12345', 'def00000'], #bank C
]

In [3]:
def get_prefixes(data, max_depth=3):
    """Extract prefixes up to depth, with counts"""
    prefixes = collections.Counter()
    for item in data:
        for i in range(1, max_depth + 1):
            prefixes[item[:i]] += 1
    return prefixes # prefixes are Counter

def add_laplace_noise(counter, epsilon=1.0, sensitivity=1):
    """Add DP noise to amounts"""
    noisy = {}
    for key, total in counter.items():
        noise = np.random.laplace(0, sensitivity/epsilon)
        noisy[key] = total + noise
    return noisy # each value of the Counter in injected with noise

In [4]:
c = get_prefixes(['abc12345', 'abc67890', 'xyz11111'])
c

Counter({'a': 2, 'ab': 2, 'abc': 2, 'x': 1, 'xy': 1, 'xyz': 1})

In [5]:
add_laplace_noise(c)

{'a': 2.6267821473516184,
 'ab': 2.75071155634096,
 'abc': 1.5610762872801338,
 'x': 1.0996505039137516,
 'xy': 2.542244594417011,
 'xyz': 0.40931378931669926}

In [6]:
# Simulate each bank building a noisy prefix tree
prefix_trees = []
for bank in banks_data:
    prefixes = get_prefixes(bank)
    noisy = add_laplace_noise(prefixes, epsilon=1.0)
    prefix_trees.append(noisy)

In [7]:
# The noise ensures the protection of the privacy
prefix_trees

[{'a': 2.3553845335234396,
  'ab': 1.2406273256972375,
  'abc': 5.2338535626171145,
  'x': 1.0542845361266635,
  'xy': 2.2602490349477216,
  'xyz': 1.2114820642115567},
 {'a': 1.3453198075599895,
  'ab': 1.8527008664315308,
  'abc': 3.7739007545402714,
  'd': 0.9443709775321596,
  'de': 2.2712232726646455,
  'def': -4.109613658772487},
 {'x': 0.3159566836142448,
  'xy': 2.835198599436423,
  'xyz': 1.6608883468910434,
  'a': -0.02575003015550248,
  'ab': 2.010618362086298,
  'abc': 1.2007825580263278,
  'd': 0.6897282371718687,
  'de': 0.3576777109494782,
  'def': 0.17431204196366834}]

In [8]:
# Aggregate (count) across all banks, 
aggregate = collections.Counter()
for tree in prefix_trees:
    for k, v in tree.items():
        aggregate[k] += v

In [9]:
aggregate

Counter({'abc': 10.208536875183713,
         'ab': 5.103946554215066,
         'xy': 5.095447634384144,
         'a': 3.6749543109279266,
         'xyz': 2.8723704111026,
         'de': 2.6289009836141237,
         'd': 1.6340992147040283,
         'x': 1.3702412197409082,
         'def': -3.935301616808818})

In [10]:
# Output top global heavy hitter prefixes
heavy_hitters = aggregate.most_common(5)
for prefix, score in heavy_hitters:
    print(f"Prefix '{prefix}' total noisy count: {score:.2f}")

Prefix 'abc' total noisy count: 10.21
Prefix 'ab' total noisy count: 5.10
Prefix 'xy' total noisy count: 5.10
Prefix 'a' total noisy count: 3.67
Prefix 'xyz' total noisy count: 2.87


#### b) Aggregate = transaction amounts

In [11]:
# Simulate per-bank transaction logs: (account_id, amount)
banks_data = [
    [('abc12345', 2000), ('abc67890', 3000), ('xyz11111', 1000)],
    [('abc12345', 1500), ('def33333', 2500), ('abc22222', 500)],
    [('abc12345', 1800), ('xyz99999', 1200), ('def00000', 800)],
]

In [12]:
def sum_amounts_by_prefix(data, max_depth=3):
    """Sum transaction amounts per prefix"""
    totals = collections.Counter()
    for acc_id, amt in data:
        for i in range(1, max_depth + 1):
            prefix = acc_id[:i]
            totals[prefix] += amt
    return totals

In [21]:
# Build noisy prefix totals per bank
prefix_trees = []
for bank in banks_data:
    prefix_totals = sum_amounts_by_prefix(bank, max_depth=8) # the values in the Counter correspond to the amounts 
    noisy = add_laplace_noise(prefix_totals, epsilon=1.0, sensitivity=0)
    prefix_trees.append(noisy)

# Aggregate globally
aggregate = collections.Counter()
for tree in prefix_trees:
    for k, v in tree.items():
        aggregate[k] += v

# Show top heavy hitters by total amount
heavy_hitters = aggregate.most_common(10)
for prefix, total in heavy_hitters:
    print(f"Prefix '{prefix}' total (noisy) amount: ${total:,.2f}")

Prefix 'a' total (noisy) amount: $8,800.00
Prefix 'ab' total (noisy) amount: $8,800.00
Prefix 'abc' total (noisy) amount: $8,800.00
Prefix 'abc1' total (noisy) amount: $5,300.00
Prefix 'abc12' total (noisy) amount: $5,300.00
Prefix 'abc123' total (noisy) amount: $5,300.00
Prefix 'abc1234' total (noisy) amount: $5,300.00
Prefix 'abc12345' total (noisy) amount: $5,300.00
Prefix 'd' total (noisy) amount: $3,300.00
Prefix 'de' total (noisy) amount: $3,300.00


## Part 2: Application to Structuring Detection

In [22]:
from collections import defaultdict

epsilon = 1.0
threshold = 10000
np.random.seed(42)

In [23]:
def generate_passport_id():
    letters = ''.join(np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), 2))
    digits = str(np.random.randint(1000000, 9999999))
    return f"{letters}{digits}"

def get_prefixes(pid, length=6):
    return [pid[:i] for i in range(2, length+1)]  # skip country code

In [24]:
# Simulated data for 3 banks
banks = []
for _ in range(3):
    bank_data = []

    # Add 50 random passports
    for _ in range(50):
        pid = generate_passport_id()
        amt = np.random.randint(100, 3000)
        bank_data.append({'passport': pid, 'amount': amt})

    # Add structuring malicious ID
    bank_data.append({'passport': 'GB0000001', 'amount': 4000})
    banks.append(bank_data)

In [25]:
# Noisy prefix trees
bank_prefix_sums = []
for bank_data in banks:
    prefix_sum = defaultdict(float)
    for txn in bank_data:
        for prefix in get_prefixes(txn['passport']):
            prefix_sum[prefix] += txn['amount']
    noisy = {k: v + np.random.laplace(0, 1000/epsilon) for k, v in prefix_sum.items()}
    bank_prefix_sums.append(noisy)

In [26]:
# Global aggregation
global_sum = defaultdict(float)
for bank in bank_prefix_sums:
    for k, v in bank.items():
        global_sum[k] += v

In [27]:
# Flagging
print("🚩 Suspicious passport prefixes (noisy sum ≥ $10,000):")
for prefix, amount in global_sum.items():
    if amount >= threshold:
        print(f" - {prefix}: ${round(amount, 2)}")

🚩 Suspicious passport prefixes (noisy sum ≥ $10,000):
 - GB: $12430.32
 - GB00: $11296.12
 - GB000: $11192.92
 - GB0000: $11879.55
